In [7]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Dense, Flatten, Conv1D, Dropout
from tensorflow.keras.layers import LeakyReLU, ReLU, TimeDistributed
from tensorflow.keras import Sequential

from utils import process_dataframe, split_train_test, scaler_function, get_X_y

In [8]:
stock = 'ASTE'
dataframe = pd.read_feather('./data/XGBoost10/df_%s_XGBoost10.feather' % (stock))

df = process_dataframe(dataframe)

X_value = pd.DataFrame(df.iloc[:, :])
y_value = pd.DataFrame(df.iloc[:, 0]) #first column is the closing price

X_train, X_test = split_train_test(X_value, train_dimension=0.8)
y_train, y_test = split_train_test(y_value.values, train_dimension=0.8)

X_scaler, y_scaler = scaler_function(X_train, y_train)
    
X_train = X_scaler.fit_transform(X_train)
y_train = y_scaler.fit_transform(y_train)
X_test = X_scaler.fit_transform(X_test)
y_test = y_scaler.fit_transform(y_test)

n_steps_in = 2
n_steps_out = 1
X_train, y_train, past_y_train = get_X_y(X_train, y_train, n_steps_in, n_steps_out)
output_dim = y_train.shape[1]
weight_initializer = RandomNormal(mean=0.00, stddev=0.02)

In [11]:
def Generator(input_dim, output_dim, feature_size, weight_initializer) -> tf.keras.models.Model:    
    model = tf.keras.Sequential()
    model.add(Conv1D(32, kernel_size=2, strides=1, 
                     padding='same',kernel_initializer= weight_initializer, 
                     batch_input_shape=(None,input_dim,feature_size)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Bidirectional(GRU(64, activation='relu', kernel_initializer= weight_initializer, return_sequences=False, 
                                 dropout=0.2, recurrent_dropout=0.0)))
    model.add(Dropout(0.2))
    model.add(Flatten())

    model.add(Dense(64, activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(32, activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    
    model.add(Dense(output_dim))
    return model

generator = Generator(X_train.shape[1], output_dim, X_train.shape[2], weight_initializer)
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 2, 32)             1248      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 2, 32)             0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              37632     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 flatten_2 (Flatten)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                      

In [12]:
def Discriminator(weight_initializer, n_steps_in, n_steps_out) -> tf.keras.models.Model:
    model = tf.keras.Sequential()
    model.add(Conv1D(32, kernel_size=2, strides=1,
                     kernel_initializer= weight_initializer, padding='same', 
                     input_shape=(n_steps_in + n_steps_out, 1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Conv1D(64, kernel_size=2, strides=1,
                     kernel_initializer= weight_initializer, padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Flatten())

    model.add(Dense(64, activation='linear', use_bias=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='linear', use_bias=True))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.2))
    
    model.add(Dense(1, activation='linear'))
    return model

discriminator = Discriminator(weight_initializer, n_steps_in, n_steps_out)
discriminator.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 3, 32)             96        
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 3, 32)             0         
                                                                 
 conv1d_5 (Conv1D)           (None, 3, 64)             4160      
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 3, 64)             0         
                                                                 
 flatten_3 (Flatten)         (None, 192)               0         
                                                                 
 dense_9 (Dense)             (None, 64)                12352     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 64)               

In [21]:
def m2tex(model):
    stringlist = []
    model.summary(line_length=90, print_fn=lambda x: stringlist.append(x))
    del stringlist[1:-4:2]
    del stringlist[-1]
    for ix in range(1,len(stringlist)-3):
        tmp = stringlist[ix]
        stringlist[ix] = tmp[0:39]+"&"+tmp[39:75]+"&"+tmp[75:]+"\\\\ \hline"
    stringlist[0] = "Model: test \\\\ \hline"
    stringlist[1] = stringlist[1]+" \hline"
    stringlist[-4] = stringlist[-4]+" \hline"
    stringlist[-3] = stringlist[-3]+" \\\\"
    stringlist[-2] = stringlist[-2]+" \\\\"
    stringlist[-1] = stringlist[-1]+" \\\\ \hline"
    prefix = ["\\begin{table}[]", "\\begin{tabular}{lll}"]
    suffix = ["\end{tabular}", "\caption{Model summary for test.}", "\label{tab:model-summary}" , "\end{table}"]
    stringlist = prefix + stringlist + suffix 
    out_str = " \n".join(stringlist)
    out_str = out_str.replace("_", "\_")
    out_str = out_str.replace("#", "\#")
    print(out_str)

In [22]:
m2tex(generator)

\begin{table}[] 
\begin{tabular}{lll} 
Model: test \\ \hline 
 Layer (type)                          & Output Shape                       & Param \#       \\ \hline \hline 
 conv1d\_3 (Conv1D)                     & (None, 2, 32)                      & 1248          \\ \hline 
 leaky\_re\_lu\_7 (LeakyReLU)             & (None, 2, 32)                      & 0             \\ \hline 
 bidirectional\_1 (Bidirectional)       & (None, 128)                        & 37632         \\ \hline 
 dropout\_2 (Dropout)                   & (None, 128)                        & 0             \\ \hline 
 flatten\_2 (Flatten)                   & (None, 128)                        & 0             \\ \hline 
 dense\_6 (Dense)                       & (None, 64)                         & 8256          \\ \hline 
 leaky\_re\_lu\_8 (LeakyReLU)             & (None, 64)                         & 0             \\ \hline 
 dense\_7 (Dense)                       & (None, 32)                         & 2080          \\

In [23]:
m2tex(discriminator)

\begin{table}[] 
\begin{tabular}{lll} 
Model: test \\ \hline 
 Layer (type)                          & Output Shape                       & Param \#       \\ \hline \hline 
 conv1d\_4 (Conv1D)                     & (None, 3, 32)                      & 96            \\ \hline 
 leaky\_re\_lu\_10 (LeakyReLU)            & (None, 3, 32)                      & 0             \\ \hline 
 conv1d\_5 (Conv1D)                     & (None, 3, 64)                      & 4160          \\ \hline 
 leaky\_re\_lu\_11 (LeakyReLU)            & (None, 3, 64)                      & 0             \\ \hline 
 flatten\_3 (Flatten)                   & (None, 192)                        & 0             \\ \hline 
 dense\_9 (Dense)                       & (None, 64)                         & 12352         \\ \hline 
 leaky\_re\_lu\_12 (LeakyReLU)            & (None, 64)                         & 0             \\ \hline 
 dropout\_3 (Dropout)                   & (None, 64)                         & 0             